In [2]:
from keras.layers.pooling import AveragePooling3D, GlobalMaxPooling3D
from keras.layers import Input, merge, Activation, Dropout
from keras.optimizers import Adamax, Adam, Nadam

In [ ]:
def compileModel(inputShape, dropRate):
    
    x = Input(inputShape)
    
    x1 = convCNNBlock(x, 8, dropRate=dropRate)
    x1Pool = AveragePooling3D(dim_ordering="th")(x)
    x1Merged = merge([x1, x1Pool], mode='concat', concat_axis=1)
    
    x2 = convCNNBlock(x1Merged, 24, dropRate=dropRate)
    x2Pool = AveragePooling3D(dim_ordering="th")(x1Pool)
    x2Merged = merge([x2, x2Pool], mode='concat', concat_axis=1)
    
    x3 = convCNNBlock(x2Merged, 64, dropRate=dropRate)
    x3Pool = AveragePooling3D(dim_ordering="th")(x2Pool)
    x3Merged = merge([x3,x3Pool], mode='concat', concat_axis=1)

    x4 = convCNNBlock(x3Merged, 72, dropRate=dropRate)
    x4Pool = AveragePooling3D(dim_ordering="th")(x3Pool)
    x4Merged = merge([x4, x4Pool], mode='concat', concat_axis=1)

    x5 = convCNNBlock(x4Merged, 72, dropRate=dropRate)
    
    xMaxPool = GlobalMaxPooling3D()(x5)
    xMaxPoolNorm = BatchNormalization()(xMaxPool) 
    
    xMalig = denseCNNBlock(xMaxPoolNorm, name='Malignancy', outSize=1, activation='sigmoid')
    xDiam = denseCNNBlock(xMaxPoolNorm, name='Diameter', outSize=1, activation='sigmoid')
    xLob = denseCNNBlock(xMaxPoolNorm, name='Lobulation', outSize=1, activation='sigmoid')
    xSpic = denseCNNBlock(xMaxPoolNorm, name='Spiculation', outSize=1, activation='sigmoid')
#     xCalc = denseCNNBlock(xMaxPoolNorm, name='Calcification', outSize=1, activation='sigmoid')
#     xSpher = denseCNNBlock(xMaxPoolNorm, name='Sphericity', outSize=1, activation='sigmoid')
    
    model = Model(input=x, output=[xMalig, xDiam, xLob, xSpic])
    
    opt = Nadam(0.01, clipvalue=1.0)
    
    print ('Compiling model...')
    
    model.compile(optimizer=opt,
                  loss={'Malignancy':'mse', 'Diameter':'mse', 'Lobulation':'mse',
                       'Spiculation':'mse'},
                  loss_weights={'Malignancy':5, 'Diameter':1, 'Lobulation':1,
                       'Spiculation':3})
    
    return model